In [1]:
pip install arcgis --no-deps

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# !pip install geopandas fiona watermark seaborn tqdm openpyxl
# !conda install krb5
# !pip install cartoframes
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.wkt import loads
import urllib.request
import zipfile
from urllib.request import urlopen
from zipfile import ZipFile
from tqdm import tqdm
import os
import sys
import shutil
import glob
import json
from datetime import datetime
import time
from collections import defaultdict
import geopandas as gpd
from geopandas import GeoDataFrame
from arcgis import GIS

In [3]:
# Get the grandparent working directory
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
datapath = f"{current_directory}\\Data"
print(datapath)

C:\Users\RamRamanathan\Documents\Projects\DOT_disadvataged\Data


In [4]:
# Run all territories
tract_numbers=['01','02','04','05','06','08','09','10','11','12','13','15','16','17','18','19','20','21','22',
               '23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42',
               '44','45','46','47','48','49','50','51','53','54','55','56','60','66','69','72','78']

In [5]:
# def download_data():
#     title_numbers = tract_numbers
    
#     for n in tqdm(title_numbers):
#         zipurl = f'https://www2.census.gov/geo/tiger/TIGER_RD18/LAYER/PLACE/tl_rd22_{n}_place.zip'
          
#         # Download the file from the URL
#         zipresp = urlopen(zipurl)
            
#         # Create a new file on the hard drive
#         tempzip = open(f'{datapath}\\Geometry_Data\\place_tmp\\tempfile.zip', "wb")
            
#         # Write the contents of the downloaded file into the new file
#         tempzip.write(zipresp.read())
            
#         # Close the newly-created file
#         tempzip.close()
            
#         # Re-open the newly-created file with ZipFile()
#         zf = ZipFile(f'{datapath}\\Geometry_Data\\place_tmp\\tempfile.zip')
            
#         # Extract its contents into <extraction_path>
#         # Note that extractall will automatically create the path
#         zf.extractall(f'{datapath}\\Geometry_Data\\place_geom\\')
            
#         # close the ZipFile instance
#         zf.close()

In [6]:
def read_shp_file():
    # get list of .shp file names in shp_folder
    shp_folder = f'{datapath}\\Geometry_Data\\place_geom'
    shp_files = glob.glob(shp_folder + "\\*.shp")
    
    # Read each shapefile into a GeoDataFrame and store in a list
    gdfs = [gpd.read_file(shp) for shp in shp_files]
    
    # Concatenate all GeoDataFrames into a single GeoDataFrame
    geom_df = gpd.pd.concat(gdfs, ignore_index=True)
#     geom_df = geom_df[['GEOID','NAME','geometry']]
    print(type(geom_df))
    geom_df = pd.DataFrame(geom_df)
    print(type(geom_df))
    return geom_df

In [7]:
# download_data()
geom_df = read_shp_file()
city_geom_df = geom_df[['STATEFP', 'NAME', 'GEOID', 'geometry']]
display(city_geom_df)

<class 'geopandas.geodataframe.GeoDataFrame'>
<class 'pandas.core.frame.DataFrame'>


,STATEFP,NAME,GEOID,geometry
0,01,Ardmore,0102260,"POLYGON ((-86.85669 34.99205, -86.85535 34.992..."
1,01,Mooresville,0151264,"POLYGON ((-86.88479 34.63009, -86.88473 34.630..."
2,01,Somerville,0171496,"MULTIPOLYGON (((-86.75696 34.44604, -86.75686 ..."
3,01,Trinity,0176872,"MULTIPOLYGON (((-87.05407 34.59983, -87.05407 ..."
4,01,Goshen,0130880,"POLYGON ((-86.13707 31.72739, -86.13669 31.727..."
...,...,...,...,...
32183,78,Coral Bay,7827000,"POLYGON ((-64.73261 18.35237, -64.73247 18.352..."
32184,78,Cruz Bay,7828000,"POLYGON ((-64.79981 18.32660, -64.79979 18.326..."
32185,78,Frederiksted Southeast,7839700,"POLYGON ((-64.88568 17.70194, -64.88555 17.702..."
32186,78,Red Hook,7865530,"POLYGON ((-64.85759 18.32304, -64.85560 18.325..."


In [8]:
community_cities = pd.read_csv("Data\\Community List.csv")
community_counties_geom = gpd.read_file("Data\\county_geometries\county_geometries.shp")

def convert_to_str_FIPS(code):
    code = str(int(code))
    
    if len(code)==2:
        return code
    return '0' + code
    
community_cities['State FP'] = community_cities['State FP'].apply(convert_to_str_FIPS)

# join to get geometries for the cities
community_cities_geom = community_cities.merge(city_geom_df, how='inner', left_on=['City', 'State FP'], right_on=['NAME', 'STATEFP'])
community_cities_geom = community_cities_geom[['City', 'State', 'geometry']]
# community_cities_geom = community_cities_geom.to_crs('WGS 84')

geometry = [loads(str(x)) for x in community_cities_geom.geometry]
community_cities_geom = community_cities_geom.drop(['geometry'], axis=1)
community_cities_geom = GeoDataFrame(community_cities_geom, crs="EPSG:4326", geometry=geometry)

In [9]:
# store with time in name to ensure unique names so that ESRI does not throw 'file exists' error
time_of_run = str(datetime.utcnow())
community_cities_geom.to_file(f'{datapath}\\Geometry_Data\\community_cities.geojson')

In [10]:
# join to get geometries for the two counties which are not cities
community_counties_geom = community_cities.merge(community_counties_geom, how='inner', left_on=['City', 'State Name'], right_on=['countyname', 'statename'])
community_counties_geom = community_counties_geom[['City', 'State', 'geometry']]

# community_cities_counties_geom = community_cities_geom.append(community_counties_geom)
# display(community_cities_geom)

,City,State,geometry
0,Boise City,ID,"MULTIPOLYGON (((-116.10184 43.55303, -116.0994..."
1,Greensboro,NC,"MULTIPOLYGON (((-79.62766 36.05635, -79.62646 ..."
2,Joplin,MO,"POLYGON ((-110.79983 48.55735, -110.79482 48.5..."
3,Philadelphia,PA,"POLYGON ((-75.28027 39.97496, -75.27951 39.975..."
4,South Bend,IN,"MULTIPOLYGON (((-86.29247 41.63576, -86.28876 ..."
5,Syracuse,NY,"POLYGON ((-76.20448 43.04538, -76.20445 43.045..."
6,Henderson,NV,"POLYGON ((-115.21250 35.89730, -115.20703 35.9..."
7,Columbus,OH,"MULTIPOLYGON (((-83.21006 40.07081, -83.21003 ..."
8,Eugene,OR,"MULTIPOLYGON (((-123.06782 44.10554, -123.0674..."
9,Lima,OH,"POLYGON ((-84.16511 40.75474, -84.16507 40.756..."


In [11]:
display(community_counties_geom)

,City,State,geometry
0,Harris,TX,"POLYGON ((-95.85685 29.87458, -95.85686 29.874..."
1,Miami-Dade,FL,"POLYGON ((-80.44093 25.95686, -80.44061 25.956..."


In [12]:
geometry = [loads(str(x)) for x in community_counties_geom.geometry]
community_counties_geom = community_counties_geom.drop(['geometry'], axis=1)
community_counties_geom = GeoDataFrame(community_counties_geom, crs="EPSG:4326", geometry=geometry)

In [13]:
community_counties_geom.to_file(f'{datapath}\\Geometry_Data\\community_counties.geojson')

In [14]:
#Log in
new_portal = GIS(username="smartbases", password="_B3$8S@NzqF7$T9")
# old_portal = GIS(username="USIgnite@bases", password="USIgnite@20036")

all_portals = [new_portal]

In [17]:
for portal in all_portals:
    print(portal)
#     # Upload mws feature layer
#     input_file_path = os.path.abspath(f'{datapath}\\Geometry_Data\\community_cities.geojson')

#     # add the geojson file as an item to portal
#     geojson_item = portal.content.add(item_properties =
#         {
#             "title": "community_cities",
#             "description": "community_cities_geom",
#             "tags": "community_cities_geom",
#             "type": "GeoJson",
#             "overwrite": True
#         },
#         data = input_file_path,
#     )

#     # publish the item to create a hosted featurelayer
#     geojson_service = geojson_item.publish(overwrite=True)

#     #Get ID and URL of the item
#     print(
#         f"New item : \n\t id : {geojson_service.id} \n\t url: {geojson_service.layers[0].url}"
#     )
    
    input_file_path = os.path.abspath(f'{datapath}\\Geometry_Data\\community_counties.geojson')

    # add the geojson file as an item to portal
    geojson_item = portal.content.add(item_properties =
        {
            "title": "community_counties",
            "description": "community_counties_geom",
            "tags": "community_counties_geom",
            "type": "GeoJson",
            "overwrite": True
        },
        data = input_file_path,
    )

    # publish the item to create a hosted featurelayer
    geojson_service = geojson_item.publish(overwrite=True)

    #Get ID and URL of the item
    print(
        f"New item : \n\t id : {geojson_service.id} \n\t url: {geojson_service.layers[0].url}"
    )

GIS @ https://US-Ignite.maps.arcgis.com version:2023.3
New item : 
	 id : e10c7702499743baa799adb3d2a0958b 
	 url: https://services5.arcgis.com/U0SnCFqVOabKg2Fv/arcgis/rest/services/community_counties/FeatureServer/0
